## Загрузка библиотек, данных

In [ ]:
!pip install -U -q PyDrive
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd

from google.colab import drive
from google.colab import files
import os

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.appName("Python Spark SQL basic example") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()

In [ ]:
# просмотр, где находится каталог с файлами на COLAB
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Предобработка данных

### viols

In [ ]:
data_viols = pd.read_excel('/content/drive/My Drive/Colab Notebooks/viols.xlsx')

In [ ]:
data_viols

,tr_viol_id,viol_code,time_check,stage_code,refuse_code,camera_id
0,12147483648,1212,2018-04-01,1,1,1
1,12147483649,129,2018-04-02,0,0,2
2,12147483650,1212,2018-04-03,1,0,3
3,12147483651,129,2018-04-04,0,1,4
4,12147483652,1212,2018-04-05,0,1,5
5,12147483653,1212,2018-04-06,0,1,6
6,12147483654,129,2018-04-21,1,0,7
7,12147483655,129,2018-04-22,1,0,8
8,12147483656,129,2018-04-23,1,2,9


In [ ]:
data_viols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   tr_viol_id    9 non-null      int64         
 1   viol_code     9 non-null      int64         
 2   time_check    9 non-null      datetime64[ns]
 3   stage_code    9 non-null      int64         
 4   refuse_code   9 non-null      int64         
 5   camera_id     9 non-null      int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 560.0 bytes


In [ ]:
data_viols.columns


Index(['tr_viol_id  ', 'viol_code   ', 'time_check  ', 'stage_code  ',
       'refuse_code ', 'camera_id   '],
      dtype='object')

In [ ]:
data_viols.rename(columns = {'tr_viol_id  ':'tr_viol_id',
                     'viol_code   ':'viol_code',
                     'time_check  ':'time_check',
                     'stage_code  ':'stage_code',
                     'refuse_code ':'refuse_code',
                     'camera_id   ':'camera_id'}, inplace = True )

In [ ]:
data_viols.columns

Index(['tr_viol_id', 'viol_code', 'time_check', 'stage_code', 'refuse_code',
       'camera_id'],
      dtype='object')

In [ ]:
#переведем столбец с датой в формат даты и времени
data_viols['time_check'] = pd.to_datetime(data_viols['time_check'], format = '%Y-%m-%d %H:%M:%S')

In [ ]:
data_viols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   tr_viol_id   9 non-null      int64         
 1   viol_code    9 non-null      int64         
 2   time_check   9 non-null      datetime64[ns]
 3   stage_code   9 non-null      int64         
 4   refuse_code  9 non-null      int64         
 5   camera_id    9 non-null      int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 560.0 bytes


In [ ]:
viols = data_viols.to_csv('viols.csv')

### opers

In [ ]:
data_opers = pd.read_excel('/content/drive/My Drive/Colab Notebooks/opers.xlsx')

In [ ]:
data_opers

,viol_oper_id,tr_viol_id,oper_code,refuse_code,isp_id,date_oper,viol_datetime
0,22147483648,12147483648,29,1,2147483648,2018-04-01,2018-04-01
1,22147483649,12147483649,31,0,2147483649,2018-04-02,2018-04-02
2,22147483650,12147483650,29,0,2147483650,2018-04-03,2018-04-03
3,22147483651,12147483651,29,1,2147483651,2018-04-04,2018-04-04
4,22147483652,12147483652,31,1,2147483652,2018-04-05,2018-04-05
5,22147483653,12147483653,31,1,2147483653,2018-04-06,2018-04-06
6,22147483654,12147483654,31,0,2147483654,2018-07-21,2018-04-21
7,22147483655,12147483655,29,0,2147483655,2018-04-22,2018-04-22
8,22147483656,12147483656,29,2,2147483656,2018-04-23,2018-04-23


In [ ]:
data_opers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   viol_oper_id       9 non-null      int64         
 1   tr_viol_id         9 non-null      int64         
 2   oper_code          9 non-null      int64         
 3   refuse_code        9 non-null      int64         
 4   isp_id             9 non-null      int64         
 5   date_oper          9 non-null      datetime64[ns]
 6   viol_datetime      9 non-null      datetime64[ns]
dtypes: datetime64[ns](2), int64(5)
memory usage: 632.0 bytes


In [ ]:
data_opers.columns

Index(['viol_oper_id', 'tr_viol_id       ', 'oper_code        ',
       'refuse_code      ', 'isp_id           ', 'date_oper        ',
       'viol_datetime    '],
      dtype='object')

In [ ]:
data_opers.rename(columns = {'tr_viol_id       ':'tr_viol_id',
                     'oper_code        ':'oper_code',
                     'refuse_code      ':'refuse_code',
                     'isp_id           ':'isp_id',
                     'date_oper        ':'date_oper',
                     'viol_datetime    ':'viol_datetime'}, inplace = True )

In [ ]:
data_opers.columns

Index(['viol_oper_id', 'tr_viol_id', 'oper_code', 'refuse_code', 'isp_id',
       'date_oper', 'viol_datetime'],
      dtype='object')

In [ ]:
#переведем столбец с датой в формат даты и времени
data_opers['date_oper'] = pd.to_datetime(data_opers['date_oper'], format = '%Y-%m-%d %H:%M:%S')
data_opers['viol_datetime'] = pd.to_datetime(data_opers['viol_datetime'], format = '%Y-%m-%d %H:%M:%S')

In [ ]:
data_opers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   viol_oper_id   9 non-null      int64         
 1   tr_viol_id     9 non-null      int64         
 2   oper_code      9 non-null      int64         
 3   refuse_code    9 non-null      int64         
 4   isp_id         9 non-null      int64         
 5   date_oper      9 non-null      datetime64[ns]
 6   viol_datetime  9 non-null      datetime64[ns]
dtypes: datetime64[ns](2), int64(5)
memory usage: 632.0 bytes


In [ ]:
opers = data_opers.to_csv('opers.csv')

## SQL - запросы

In [ ]:
# загрузка данных
viols = spark.read.load('/content/drive/My Drive/Colab Notebooks/viols.csv',
                       format='csv', header='true', inferSchema='true').createOrReplaceTempView('viols')
opers = spark.read.load('/content/drive/My Drive/Colab Notebooks/opers.csv',
                       format='csv', header='true', inferSchema='true').createOrReplaceTempView('opers')

In [ ]:
spark.sql(
    "SELECT *\
    FROM opers"
    ).show()

+---+------------+-----------+---------+-----------+----------+-------------------+-------------------+
|_c0|viol_oper_id| tr_viol_id|oper_code|refuse_code|    isp_id|          date_oper|      viol_datetime|
+---+------------+-----------+---------+-----------+----------+-------------------+-------------------+
|  0| 22147483648|12147483648|       29|          1|2147483648|2018-04-01 00:00:00|2018-04-01 00:00:00|
|  1| 22147483649|12147483649|       31|          0|2147483649|2018-04-02 00:00:00|2018-04-02 00:00:00|
|  2| 22147483650|12147483650|       29|          0|2147483650|2018-04-03 00:00:00|2018-04-03 00:00:00|
|  3| 22147483651|12147483651|       29|          1|2147483651|2018-04-04 00:00:00|2018-04-04 00:00:00|
|  4| 22147483652|12147483652|       31|          1|2147483652|2018-04-05 00:00:00|2018-04-05 00:00:00|
|  5| 22147483653|12147483653|       31|          1|2147483653|2018-04-06 00:00:00|2018-04-06 00:00:00|
|  6| 22147483654|12147483654|       31|          0|2147483654|2

In [ ]:
spark.sql(
    "SELECT *\
    FROM viols"
    ).show()

+---+-----------+---------+-------------------+----------+-----------+---------+
|_c0| tr_viol_id|viol_code|         time_check|stage_code|refuse_code|camera_id|
+---+-----------+---------+-------------------+----------+-----------+---------+
|  0|12147483648|     1212|2018-04-01 00:00:00|         1|          1|        1|
|  1|12147483649|      129|2018-04-02 00:00:00|         0|          0|        2|
|  2|12147483650|     1212|2018-04-03 00:00:00|         1|          0|        3|
|  3|12147483651|      129|2018-04-04 00:00:00|         0|          1|        4|
|  4|12147483652|     1212|2018-04-05 00:00:00|         0|          1|        5|
|  5|12147483653|     1212|2018-04-06 00:00:00|         0|          1|        6|
|  6|12147483654|      129|2018-04-21 00:00:00|         1|          0|        7|
|  7|12147483655|      129|2018-04-22 00:00:00|         1|          0|        8|
|  8|12147483656|      129|2018-04-23 00:00:00|         1|          2|        9|
+---+-----------+---------+-

### 1. Дата нарушения ПДД & Число нарушений ПДД всего

In [ ]:
%%time
spark.sql(
    "SELECT time_check,\
    COUNT(tr_viol_id) AS total_fines\
    FROM viols\
    WHERE (CAST(time_check AS date) BETWEEN '2018-04-01' AND '2018-04-20')\
    GROUP BY time_check"
    ).show()

+-------------------+-----------+
|         time_check|total_fines|
+-------------------+-----------+
|2018-04-04 00:00:00|          1|
|2018-04-06 00:00:00|          1|
|2018-04-05 00:00:00|          1|
|2018-04-01 00:00:00|          1|
|2018-04-03 00:00:00|          1|
|2018-04-02 00:00:00|          1|
+-------------------+-----------+

CPU times: user 3.41 ms, sys: 0 ns, total: 3.41 ms
Wall time: 272 ms


### 2. Дата нарушения ПДД & Число нарушений ПДД у которых результирующий код обработки нарушения ПДД равен 0

In [ ]:
%%time
spark.sql(
    "SELECT viol_datetime,\
    COUNT(tr_viol_id) AS count_f_refuse_code_0\
    FROM opers\
    WHERE (CAST(viol_datetime AS date) BETWEEN '2018-04-01' AND '2018-04-20')\
    AND refuse_code = 0\
    GROUP BY viol_datetime"
    ).show()

+-------------------+---------------------+
|      viol_datetime|count_f_refuse_code_0|
+-------------------+---------------------+
|2018-04-03 00:00:00|                    1|
|2018-04-02 00:00:00|                    1|
+-------------------+---------------------+

CPU times: user 7.39 ms, sys: 82 µs, total: 7.47 ms
Wall time: 481 ms


### 3. Дата нарушения ПДД & Число нарушений ПДД по которым была операция обработки тип 29

In [ ]:
%%time
spark.sql(
    "SELECT viol_datetime,\
    COUNT(tr_viol_id) AS count_f_oper_code_29\
    FROM opers_v3\
    WHERE (CAST(viol_datetime AS date) BETWEEN '2018-04-01' AND '2018-04-20')\
    AND oper_code = 29\
    GROUP BY viol_datetime"
    ).show()

+-------------------+--------------------+
|      viol_datetime|count_f_oper_code_29|
+-------------------+--------------------+
|2018-04-04 00:00:00|                   1|
|2018-04-01 00:00:00|                   1|
|2018-04-03 00:00:00|                   1|
+-------------------+--------------------+

CPU times: user 6.38 ms, sys: 443 µs, total: 6.82 ms
Wall time: 424 ms


### 4. Дата нарушения ПДД & Число нарушений ПДД по которым не было операции обработки тип 29, а результирующий код обработки нарушения ПДД равен 0

In [ ]:
%%time
spark.sql(
    "SELECT viol_datetime,\
    COUNT(tr_viol_id) AS count_f_code_0_and_29\
    FROM opers_v3\
    WHERE (CAST(viol_datetime AS date) BETWEEN '2018-04-01' AND '2018-04-20')\
    AND refuse_code = 0\
    AND oper_code = 29\
    GROUP BY viol_datetime"
    ).show()

+-------------------+---------------------+
|      viol_datetime|count_f_code_0_and_29|
+-------------------+---------------------+
|2018-04-03 00:00:00|                    1|
+-------------------+---------------------+

CPU times: user 4.91 ms, sys: 1.17 ms, total: 6.08 ms
Wall time: 410 ms


### 5. Сводная таблица

In [ ]:
%%time
spark.sql(
    "WITH\
    s1 AS (SELECT time_check,\
    COUNT(tr_viol_id) AS total_fines\
    FROM viols\
    WHERE (CAST(time_check AS date) BETWEEN '2018-04-01' AND '2018-04-20')\
    GROUP BY time_check),\
    \
    s2 AS (SELECT viol_datetime,\
    COUNT(tr_viol_id) AS count_f_refuse_code_0\
    FROM opers\
    WHERE (CAST(viol_datetime AS date) BETWEEN '2018-04-01' AND '2018-04-20')\
    AND refuse_code = 0\
    GROUP BY viol_datetime),\
    \
    s3 AS (SELECT viol_datetime,\
    COUNT(tr_viol_id) AS count_f_oper_code_29\
    FROM opers_v3\
    WHERE (CAST(viol_datetime AS date) BETWEEN '2018-04-01' AND '2018-04-20')\
    AND oper_code = 29\
    GROUP BY viol_datetime),\
    \
    s4 AS (SELECT viol_datetime,\
    COUNT(tr_viol_id) AS count_f_code_0_and_29\
    FROM opers_v3\
    WHERE (CAST(viol_datetime AS date) BETWEEN '2018-04-01' AND '2018-04-20')\
    AND refuse_code = 0\
    AND oper_code = 29\
    GROUP BY viol_datetime)\
    \
    SELECT s1.time_check,\
    s1.total_fines,\
    s2.count_f_refuse_code_0,\
    s3.count_f_oper_code_29,\
    s4.count_f_code_0_and_29 \
    FROM s1\
    LEFT OUTER JOIN s2 ON s1.time_check=s2.viol_datetime\
    LEFT OUTER JOIN s3 ON s1.time_check=s3.viol_datetime\
    LEFT OUTER JOIN s4 ON s1.time_check=s4.viol_datetime"
    ).show()

+-------------------+-----------+---------------------+--------------------+---------------------+
|         time_check|total_fines|count_f_refuse_code_0|count_f_oper_code_29|count_f_code_0_and_29|
+-------------------+-----------+---------------------+--------------------+---------------------+
|2018-04-04 00:00:00|          1|                 null|                   1|                 null|
|2018-04-06 00:00:00|          1|                 null|                null|                 null|
|2018-04-05 00:00:00|          1|                 null|                null|                 null|
|2018-04-01 00:00:00|          1|                 null|                   1|                 null|
|2018-04-03 00:00:00|          1|                    1|                   1|                    1|
|2018-04-02 00:00:00|          1|                    1|                null|                 null|
+-------------------+-----------+---------------------+--------------------+---------------------+

CPU times